# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\incorrectdata.csv")

In [ ]:
# df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
# df.head(10)

In [3]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,20-05-2024,MWN9Y1D,NCR North,Sander Maghirang
1,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,10-05-2024,M7V4Z2N,NCR South 2,Grace Belarmino
2,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,30-05-2024,MQ5R16Y,South Luzon 1 (A),Aubrey Lalong Isip
3,JSL,SHAR Novaliches,Standalone,POS Operational,26-06-2024,M0C5496,NCR North,Sander Maghirang
4,Home Along,Home Along Bongabon,Standalone,POS Operational,07-06-2024,MTKYL7W,North Luzon 2,Rachelle Belmonte


In [4]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['BrandStores', 'POS', 'StoreLocation', 'Status', 'DateOpened',
       'PurpleKey', 'Region', 'TSM'],
      dtype='object')

In [5]:
df['BrandStores'].value_counts()

BrandStores
Home Along     3
Homeworks      2
Workstation    1
JSL            1
Rulls          1
Name: count, dtype: int64

In [6]:
df['PURPLE KEY'].value_counts()

KeyError: 'PURPLE KEY'

In [7]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

KeyError: "['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION'] not in index"

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    8 non-null      object
 1   POS            8 non-null      object
 2   StoreLocation  8 non-null      object
 3   Status         8 non-null      object
 4   DateOpened     8 non-null      object
 5   PurpleKey      8 non-null      object
 6   Region         8 non-null      object
 7   TSM            8 non-null      object
dtypes: object(8)
memory usage: 640.0+ bytes


In [9]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [20]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,20-05-2024,MWN9Y1D,NCR North,Sander Maghirang
1,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,10-05-2024,M7V4Z2N,NCR South 2,Grace Belarmino
2,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,30-05-2024,MQ5R16Y,South Luzon 1 (A),Aubrey Lalong Isip
3,JSL,SHAR Novaliches,Standalone,POS Operational,26-06-2024,M0C5496,NCR North,Sander Maghirang
4,Home Along,Home Along Bongabon,Standalone,POS Operational,07-06-2024,MTKYL7W,North Luzon 2,Rachelle Belmonte


In [22]:
from datetime import datetime
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%m-%Y')
# Format the datetime object to YYYY-MM-DD
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')

In [10]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


ValueError: time data "20-05-2024" doesn't match format "%B %d, %Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [19]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

object
20-05-2024


AttributeError: Can only use .dt accessor with datetimelike values

In [12]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "20-05-2024" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
print(df['DateOpened'].dtype)
# datetime64[ns]

object


In [14]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

20-05-2024


In [25]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [26]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    8 non-null      object        
 1   POS            8 non-null      object        
 2   StoreLocation  8 non-null      object        
 3   Status         8 non-null      object        
 4   DateOpened     8 non-null      datetime64[ns]
 5   PurpleKey      8 non-null      object        
 6   Region         8 non-null      object        
 7   TSM            8 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 640.0+ bytes


In [28]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,2024-05-20,MWN9Y1D,NCR North,Sander Maghirang
1,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,2024-05-10,M7V4Z2N,NCR South 2,Grace Belarmino
2,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,2024-05-30,MQ5R16Y,South Luzon 1 (A),Aubrey Lalong Isip
3,JSL,SHAR Novaliches,Standalone,POS Operational,2024-06-26,M0C5496,NCR North,Sander Maghirang
4,Home Along,Home Along Bongabon,Standalone,POS Operational,2024-06-07,MTKYL7W,North Luzon 2,Rachelle Belmonte


In [29]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [30]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e69bd5c3-34ba-4733-b7f9-9e3225d3bdae>

In [31]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID e69cca32-612d-43af-93d5-e9797ad6fe53 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(685, 8)


In [32]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 38779c78-0b45-4589-8b50-b7b82c1409fd successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [33]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
ArkHome               4
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Gadgetmate           10
Hardware Sugar        2
Hazrat                6
Home Along           91
Home Factory          2
Homeworks            13
JSL                   6
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Presnet              21
Prince and Aviel      3
RCB Mobile            3
Rosa Mobile           9
Rulls               161
Sogo Home             3
Willy and Sons       29
Workstation           6
Wyse                  1
Xtreme               23
Name: count, dtype: int64

In [34]:
dfsil.shape

(685, 8)

In [35]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 693 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [36]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 83bab9ae-7b11-49ee-bdf7-bf58aa7554bf successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (693, 8)


In [37]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
683,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
684,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto
685,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,2024-05-20,MWN9Y1D,NCR North,Sander Maghirang
686,JSL,SHAR Novaliches,Standalone,POS Operational,2024-06-26,M0C5496,NCR North,Sander Maghirang
687,Home Along,Home Along Bongabon,Standalone,POS Operational,2024-06-07,MTKYL7W,North Luzon 2,Rachelle Belmonte
688,Home Along,Home Along Balanga 1,Standalone,POS Operational,2024-06-13,MHDTX6A,North Luzon 2,Rachelle Belmonte
689,Home Along,Home Along Floridablanca,Standalone,POS Operational,2024-06-21,MVZS5FA,North Luzon 2,Rachelle Belmonte
690,Rulls,Realme Serin,Ayala Mall Serin,POS Operational,2024-06-01,M87F26K,South Luzon 1 (A),Aubrey Lalong Isip
691,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,2024-05-30,MQ5R16Y,South Luzon 1 (A),Aubrey Lalong Isip
692,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,2024-05-10,M7V4Z2N,NCR South 2,Grace Belarmino


In [38]:
df['BrandStores'].value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
ArkHome               4
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Gadgetmate           10
Hardware Sugar        2
Hazrat                6
Home Along           94
Home Factory          2
Homeworks            15
JSL                   7
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Presnet              21
Prince and Aviel      3
RCB Mobile            3
Rosa Mobile           9
Rulls               162
Sogo Home             3
Willy and Sons       29
Workstation           7
Wyse                  1
Xtreme               23
Name: count, dtype: int64

In [39]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,1,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
1,2,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
14,649,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
13,645,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
15,654,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
16,655,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
5,566,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
6,568,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
11,585,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo
12,586,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo


In [40]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [41]:
df[df['PurpleKey']== 'MWN9Y1D']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
685,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,2024-05-20,MWN9Y1D,NCR North,Sander Maghirang


In [42]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=60fd6f7f-103c-4dc3-8210-93aff79437b0>

In [43]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6975412c-7c6f-4df7-99e0-38cb7557d14b>

In [44]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=172d99d7-76dd-4d01-a7a0-c69d5d181ab5>

In [45]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID 5565c9cc-8068-4613-a37b-b0a423e413ae successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(693, 8)

In [46]:
df[df['BrandStores']=='Hazrat'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
87,Hazrat,Hazrat Multibrand 3,Iba Town Center,For Opening,2024-07-09,MZNOY5K,Rachelle Belmonte,North Luzon 2
88,Hazrat,Infinix Kiosk Iba,Iba Town Center,For Opening,2024-07-09,MU1BWH7,Rachelle Belmonte,North Luzon 2
89,Hazrat,Tecno Kiosk Iba,Iba Town Center,For Opening,2024-07-09,MCRULIH,Rachelle Belmonte,North Luzon 2
90,Hazrat,Hazrat Multibrand 2,Iba Town Center,For Opening,2024-07-09,M9G7KT2,Rachelle Belmonte,North Luzon 2
91,Hazrat,Hazrat Multibrand 1,Iba Town Center,For Opening,2024-07-09,M4YP4VS,Rachelle Belmonte,North Luzon 2
92,Hazrat,RealMe Kiosk Iba,Iba Town Center,For Opening,2024-07-09,M7BI4H3,Rachelle Belmonte,North Luzon 2


In [47]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
1,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
2,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
3,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
4,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
5,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
6,Rulls,Mobile Craze PG Tanay Rizal,Puregold Tanay Rizal,For Opening,2024-04-15,MFCWRJQ,NCR East,Albee David
7,Rulls,Mobilecraze K1 PG Marikina,Puregold Marikina,For Opening,2024-04-15,M9KLTD1,NCR East,Albee David
8,Rulls,Realme Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MBLR71F,NCR East,Albee David
9,MemoXpress,Memoxpress Sta Lucia East Grand Mall,Sta Lucia Mall,For Opening,2024-04-15,MRTAIQG,NCR East,Albee David


In [48]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
87,Hazrat,Hazrat Multibrand 3,Iba Town Center,For Opening,2024-07-09,MZNOY5K,Rachelle Belmonte,North Luzon 2
88,Hazrat,Infinix Kiosk Iba,Iba Town Center,For Opening,2024-07-09,MU1BWH7,Rachelle Belmonte,North Luzon 2
89,Hazrat,Tecno Kiosk Iba,Iba Town Center,For Opening,2024-07-09,MCRULIH,Rachelle Belmonte,North Luzon 2
90,Hazrat,Hazrat Multibrand 2,Iba Town Center,For Opening,2024-07-09,M9G7KT2,Rachelle Belmonte,North Luzon 2
91,Hazrat,Hazrat Multibrand 1,Iba Town Center,For Opening,2024-07-09,M4YP4VS,Rachelle Belmonte,North Luzon 2
92,Hazrat,RealMe Kiosk Iba,Iba Town Center,For Opening,2024-07-09,M7BI4H3,Rachelle Belmonte,North Luzon 2
124,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno
122,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno
123,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
251,JSL,JSL Novaliches,Standalone,For Opening,2024-06-26,M65WJCU,NCR North,Sander Maghirang


In [49]:
df[df['PurpleKey'] =='MWN9Y1D']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
250,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,2024-05-20,MWN9Y1D,NCR North,Sander Maghirang


In [50]:
df[df['BrandStores']=='Hazrat']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
87,Hazrat,Hazrat Multibrand 3,Iba Town Center,For Opening,2024-07-09,MZNOY5K,Rachelle Belmonte,North Luzon 2
88,Hazrat,Infinix Kiosk Iba,Iba Town Center,For Opening,2024-07-09,MU1BWH7,Rachelle Belmonte,North Luzon 2
89,Hazrat,Tecno Kiosk Iba,Iba Town Center,For Opening,2024-07-09,MCRULIH,Rachelle Belmonte,North Luzon 2
90,Hazrat,Hazrat Multibrand 2,Iba Town Center,For Opening,2024-07-09,M9G7KT2,Rachelle Belmonte,North Luzon 2
91,Hazrat,Hazrat Multibrand 1,Iba Town Center,For Opening,2024-07-09,M4YP4VS,Rachelle Belmonte,North Luzon 2
92,Hazrat,RealMe Kiosk Iba,Iba Town Center,For Opening,2024-07-09,M7BI4H3,Rachelle Belmonte,North Luzon 2


In [51]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)